In [2]:
### created and distributed by LLNL libROM team
### modified by Suparno Bhattacharyya

# Generate Snapshots for MOR

## Imports and custom class definitions

In [3]:
# Standard library imports for operating system interaction, file input/output, and system specifics
import os
import io
import pathlib
import sys

# Attempt to import the parallel version of mfem (PyMFEM). If unsuccessful, provide instructions for installation.
try:
    import mfem.par as mfem
except ModuleNotFoundError:
    msg = "PyMFEM is not installed yet. Install PyMFEM:\n"
    msg += "\tgit clone https://github.com/mfem/PyMFEM.git\n"
    msg += "\tcd PyMFEM\n"
    msg += "\tpython3 setup.py install --with-parallel\n"
    raise ModuleNotFoundError(msg)

# Import specific functionalities from mfem after ensuring it is installed
from mfem.par import intArray
from ctypes import c_double

# Imports for handling file paths in a way that's independent of the user's operating system
from os.path import expanduser, join, dirname

# Import NumPy for numerical computations and specific mathematical functions
import numpy as np
from numpy import sin, cos, exp, sqrt, pi, abs, array, floor, log, sum

# Extend the system path to include the build directory for local module imports
sys.path.append("../../build")

# Import specific algorithms and linear algebra functionalities from pylibROM
import pylibROM.algo as algo
import pylibROM.linalg as libROM
from pylibROM.mfem import ComputeCtAB

# Import utility functions from pylibROM, such as a stopwatch for timing
from pylibROM.python_utils import StopWatch

In [4]:
from IPython.display import Image, display


## Build the Finite Element Model in MFEM

#### By construct pylibROM is capable of handling parallel computations

In [5]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
myid = comm.Get_rank()
num_procs = comm.Get_size()

#### Specifying arguments for MFEM

In [6]:
from parser_config import get_parser
parser = get_parser()

#### Parameters

In [7]:
frequency = 1.0
id = 0
phase = 1

In [8]:
if phase == 1:
    data_dir = "training_data/"
else:
    data_dir = "rom_data/"

In [9]:
args = parser.parse_args(f"-offline -f {frequency} -id {id}".split())

parser.print_options(args)

mesh_file = os.path.abspath(os.path.join('', args.mesh))
freq            = args.frequency
fom             = args.fom
offline         = args.offline
online          = args.online
merge           = args.merge
device_config   = args.device
id              = args.id
order           = args.order
nsets           = args.nset
coef            = args.coefficient
pa              = args.partial_assembly
static_cond     = args.static_condensation
visualization   = args.visualization
precision       = 8
paraview        = args.paraview

kappa = freq * np.pi

Options used:
   --mesh  star.mesh
   --order  1
   --id  0
   --nset  0
   --static_condensation  False
   --partial_assembly  False
   --frequency  1.0
   --coefficient  1.0
   --device  cpu
   --visit_datafiles  False
   --visualization  True
   --fom  False
   --offline  True
   --online  False
   --merge  False
   --paraview  True


In [10]:
import os
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [11]:
max_num_snapshots = 100
update_right_SV = False
isIncremental = False
basisName = data_dir+"basis"
basisFileName = "%s%d" % (basisName, id)
solveTimer, assembleTimer, mergeTimer = StopWatch(), StopWatch(), StopWatch()

#### Enable hardware devices such as GPUs, and programming models such as CUDA, OCCA, RAJA and OpenMP based on command line options.

In [12]:
device = mfem.Device(device_config)
if (myid == 0):
    device.Print()

Device configuration: cpu
Memory configuration: host-std


#### Read the (serial) mesh from the given mesh file on all processors.  We can handle triangular, quadrilateral, tetrahedral, hexahedral, surface and volume meshes with the same code.**

<img src="./Mesh_view/mesh.png" alt="Getting started" />      <img src="./Mesh_view/meshgrid.png" alt="Getting started" height="430" />

In [13]:
# mesh_file = 'Mesh/astructured_rectangular.msh'
# mesh_file = 'Mesh/TAM_mesh/transformed_atm.vtu'
mesh = mfem.Mesh(mesh_file, 0, 0)
# The code mesh = mfem.Mesh(mesh_file, 1, 1) initializes an MFEM mesh object from a file named mesh_file. 
# The first 1 indicates that edges should be generated for the mesh elements, which is necessary for certain types of simulations. 
# The second 1 specifies that the mesh should be refined once upon loading.

dim = mesh.Dimension()

#### Refine the serial mesh on all processors to increase the resolution. In this example we do 'ref_levels' of uniform refinement. We choose 'ref_levels' to be the largest number that gives a final mesh with no more than 10,000 elements.

In [ ]:
ref_levels = int(np.floor(np.log(10000. / mesh.GetNE()) / log(2.) / dim))
for l in range(ref_levels):
    mesh.UniformRefinement()

#### Define a parallel mesh by a partitioning of the serial mesh. Refine this mesh further in parallel to increase the resolution. Once the parallel mesh is defined, the serial mesh can be deleted.

In [ ]:
pmesh = mfem.ParMesh(comm, mesh)
mesh.Clear()
par_ref_levels = 2
for l in range(par_ref_levels):
    pmesh.UniformRefinement()

#### Define a parallel finite element space on the parallel mesh. Here we use continuous Lagrange finite elements of the specified order. If order < 1, we instead use an isoparametric/isogeometric space.

In [ ]:
if (order > 0):
    fec = mfem.H1_FECollection(order, dim)
    delete_fec = True
elif (pmesh.GetNodes()):
    fec = pmesh.GetNodes().OwnFEC()
    delete_fec = False
    if (myid == 0):
        print(f"Using isoparametric FEs: {fec.Name()}")
else:
    fec = mfem.H1_FECollection(1, dim)
    delete_fec = True

fespace = mfem.ParFiniteElementSpace(pmesh, fec)
size = fespace.GlobalTrueVSize()
if (myid == 0):
    print(f"Number of finite element unknowns: {size}")

Number of finite element unknowns: 253727


#### Determine the list of true (i.e. parallel conforming) essential   boundary dofs. In this example, the boundary conditions are defined   by marking all the boundary attributes from the mesh as essential   (Dirichlet) and converting them to a list of true dofs.

In [ ]:
ess_tdof_list = mfem.intArray()
if (pmesh.bdr_attributes.Size() > 0):
    ess_bdr = mfem.intArray(pmesh.bdr_attributes.Max())
    ess_bdr.Assign(1)
    fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)

#### Set BasisGenerator if offline


In [ ]:
options = libROM.Options(fespace.GetTrueVSize(), max_num_snapshots, 1, update_right_SV)
generator = libROM.BasisGenerator(options, isIncremental, basisFileName)

#### Set up the parallel linear form b(.) which corresponds to the right-hand side of the FEM linear system, which in this case is (f,phi_i) where f is given by the function f_exact and phi_i are the basis functions in the finite element fespace.

In [ ]:
class RightHandSide(mfem.PyCoefficient):
    def __init__(self, kappa):
        mfem.PyCoefficient.__init__(self)
        self.kappa = kappa
        
    def EvalValue(self, x):
        if dim == 3:
            return sin(self.kappa * (x[0] + x[1] + x[2]))
        else:
            return sin(self.kappa * (x[0] + x[1]))

In [ ]:
assembleTimer.Start()
b = mfem.ParLinearForm(fespace)
f = RightHandSide(kappa)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(f))
b.Assemble()

#### Define the solution vector x as a parallel finite element grid function corresponding to fespace. Initialize x with initial guess of zero, which satisfies the boundary conditions.

In [ ]:
x = mfem.ParGridFunction(fespace)
x.Assign(0.0)

<mfem._par.pgridfunc.ParGridFunction; proxy of <Swig Object of type 'mfem::ParGridFunction *' at 0x7f366ac73de0> >

#### Set up the parallel bilinear form a(.,.) on the finite element space corresponding to the Laplacian operator -Delta, by adding the Diffusion domain integrator.

In [ ]:
a = mfem.ParBilinearForm(fespace)
one = mfem.ConstantCoefficient(coef)
if (pa):
    a.SetAssemblyLevel(mfem.AssemblyLevel_PARTIAL)
a.AddDomainIntegrator(mfem.DiffusionIntegrator(one))

#### Assemble the parallel bilinear form and the corresponding linear system, applying any necessary transformations such as: parallel  assembly, eliminating boundary conditions, applying conforming  constraints for non-conforming AMR, static condensation, etc.

In [ ]:
if (static_cond):
    a.EnableStaticCondensation()
a.Assemble()

# A = mfem.OperatorPtr()
A = mfem.HypreParMatrix()
B = mfem.Vector()
X = mfem.Vector()
a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)
assembleTimer.Stop()

#### The offline phase -Solve the full order linear system A X = B

In [ ]:
if (fom or offline):
    # 17. Solve the full order linear system A X = B
    prec = None
    if pa:
        if mfem.UsesTensorBasis(fespace):
            prec = mfem.OperatorJacobiSmoother(a, ess_tdof_list)
    else:
        prec = mfem.HypreBoomerAMG(A)

    cg = mfem.CGSolver(comm)
    cg.SetRelTol(1e-12)
    cg.SetMaxIter(2000)
    cg.SetPrintLevel(1)
    if (prec is not None):
        cg.SetPreconditioner(prec)
    # cg.SetOperator(A.Ptr())
    cg.SetOperator(A)
    solveTimer.Start()
    cg.Mult(B, X)
    solveTimer.Stop()
    if (prec is not None):
        del prec

    # 18. take and write snapshot for ROM
    if (offline):
        # NOTE: mfem Vector::GetData returns a SWIG Object of type double *.
        # To make it compatible with pybind11, we use ctypes to read data from the memory address.
        xData = np.array((c_double * X.Size()).from_address(int(X.GetData())), copy=False) # this does not copy the data.
        addSample = generator.takeSample(xData, 0.0, 0.01)
        generator.writeSnapshot()
        del generator
        del options



 Num MPI tasks = 1

 Num OpenMP threads = 1


BoomerAMG SETUP PARAMETERS:

 Max levels = 25
 Num levels = 7

 Strength Threshold = 0.250000
 Interpolation Truncation Factor = 0.000000
 Maximum Row Sum Threshold for Dependency Weakening = 0.900000

 Coarsening Type = HMIS 

 No. of levels of aggressive coarsening: 1

 Interpolation on agg. levels= multipass interpolation
 measures are determined locally


 No global partition option chosen.

 Interpolation = extended+i interpolation

Operator Matrix Information:

             nonzero            entries/row          row sums
lev    rows  entries sparse   min  max     avg      min         max
  0  253727  2264543  0.000     4   13     8.9  -1.404e-02   4.454e+00
  1   15324   129732  0.001     3   12     8.5  -1.278e-02   6.158e+00
  2    4990    97420  0.004     4   29    19.5  -8.590e-15   8.145e+00
  3    1355    32665  0.018     4   37    24.1  -1.062e-14   6.910e+00
  4     300     6168  0.069     5   36    20.6   5.563e-05   6.626

#### Recover the parallel grid function corresponding to X. This is the  local finite element solution on each processor

In [ ]:
a.RecoverFEMSolution(X, b, x)

#### Stringstream sol_dofs_name, sol_dofs_name_fom

In [ ]:
sol_dofs_name = data_dir+f"sol_{id}_"+"dofs_fom.%06d" % myid

#### Save the refined mesh and the solution in parallel. This output can be viewed later using GLVis: "glvis -np <np> -m mesh -g sol"

In [ ]:
mesh_name  = data_dir+"mesh.%06d" % myid
sol_name = data_dir+f"sol_{id}"+".%06d" % myid

pmesh.Print(mesh_name, precision)

output = io.StringIO()
output.precision = precision
x.Save(output)

# with open(sol_name, 'wb') as file:
#     file.write(output.getvalue())

fid = open(sol_name, 'w')
fid.write(output.getvalue())
fid.close()

xData = np.array((c_double * X.Size()).from_address(int(X.GetData())), copy=False)
np.savetxt(sol_dofs_name, xData, fmt='%.16f')

#### Save data in the ParaView format

In [ ]:
if paraview:
    paraview_dc = mfem.ParaViewDataCollection(f"Poisson_pv_{id}", pmesh)
        
    paraview_dc.SetPrefixPath(data_dir+"ParaView")
    paraview_dc.SetLevelsOfDetail(order)
    paraview_dc.SetCycle(0)
    paraview_dc.SetDataFormat(mfem.VTKFormat_BINARY)
    paraview_dc.SetHighOrderOutput(True)
    paraview_dc.SetTime(0.0)
    paraview_dc.RegisterField(f"solution_{id}", x)
    paraview_dc.Save()

#### Save data file

In [ ]:
# Define the file name where you want to write the times
filename_FOM = data_dir+"Assemble_solve_FOM_log.csv"

if offline:
    if myid == 0:
        # Open the file in append mode
        if fom or offline:
            with open(filename_FOM, 'a') as file:
                # Write assembly and solve times for FOM to the file in a structured format
                file.write("FOM,Assemble,%e\n" % assembleTimer.duration)
                file.write("FOM,Solve,%e\n" % solveTimer.duration)

#### Free the used memory

In [ ]:
if (delete_fec):
    del fec
MPI.Finalize()